In [119]:
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
DATASET_ID = 'gcp_workshop'
bucket_name = 'gcp-workshop-input'
REGION = 'europe-west4'

### load data in GCS bucket

### list file in input bucket

In [41]:
from google.cloud import storage

storage_client = storage.Client()

bucket_name = 'gcp-workshop-input'
blobs = storage_client.list_blobs(bucket_name)

# Note: The call returns a response only when the iterator is consumed.
for blob in blobs:
    print(blob.name)

auto_clean_train.csv
test_auto.csv
train_auto.csv


### Read with pandas

In [26]:
#read data from GCS
import pandas as pd
df_train = pd.read_csv("gs://gcp-workshop-input/train_auto.csv")
df_test = pd.read_csv("gs://gcp-workshop-input/train_auto.csv")

### bQ client

In [9]:
from google.cloud import bigquery
# Construct a BigQuery client object.
client = bigquery.Client(project= PROJECT_ID)

### DATASET CREATION

In [49]:
#dataset is the bq database, inside that, we will find the single table
# Construct a full Dataset object to send to the API.

dataset = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_ID}")
dataset.location = "EU"

# Send the dataset to the API for creation, with an explicit timeout.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
dataset = client.create_dataset(dataset = dataset, timeout=30)  # Make an API request.
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset sandbox-mvp-001.gcp_workshop


### LOAD DATA FROM GCS TO BQ

In [50]:
TABLE_ID = 'train'
table_id = f"{client.project}.{dataset.dataset_id}.{TABLE_ID}"

job_config = bigquery.LoadJobConfig(autodetect=True,
                                    skip_leading_rows=1,
                                    source_format=bigquery.SourceFormat.CSV,
                                   )

file_name = "train_auto.csv"
uri = f"gs://{bucket_name}/{file_name}"

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 8161 rows.


### Access data from BQ

In [174]:
PROJECT_ID, DATASET_ID, TABLE_ID = 'sandbox-mvp-001', 'gcp_workshop', 'train' 

In [175]:
print(PROJECT_ID, DATASET_ID, TABLE_ID)

sandbox-mvp-001 gcp_workshop train


In [176]:
from google.cloud import bigquery
# Construct a BigQuery client object.
client = bigquery.Client(project= PROJECT_ID)

In [177]:
read_query = f"SELECT * FROM {PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"
df = client.query(read_query).to_dataframe()

In [178]:
df.head()

,INDEX,TARGET_FLAG,TARGET_AMT,KIDSDRIV,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,...,BLUEBOOK,TIF,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CAR_AGE,URBANICITY
0,46,0,0.0,0,43,2,17,145353,False,418785,...,29450,6,Van,False,0,0,False,0,15,Highly Urban/ Urban
1,155,0,0.0,0,33,1,12,111427,False,321483,...,28180,1,Van,False,0,0,False,3,4,Highly Urban/ Urban
2,430,1,1444.0,0,32,1,12,76985,True,0,...,19500,7,Van,True,6762,1,False,4,15,Highly Urban/ Urban
3,474,0,0.0,0,45,1,8,45105,False,181896,...,17540,14,Van,True,36562,3,True,4,9,Highly Urban/ Urban
4,637,0,0.0,1,42,3,13,58797,True,0,...,19980,4,Van,True,1310,2,False,4,23,Highly Urban/ Urban


### Preprocess data

In [4]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# This object performs several preprocessing of the data set

class Preprocessor: 

    def __init__(self, df):
        Preprocessor.data = df.copy()
        Preprocessor.processed_data = None

    def dropVar(self, vars):
        # Remove a variable from the dataset
        self.processed_data = self.data.drop(vars, axis=1)

    def str2int(self, variables):
        # Define a function that extract the numerical value from a string like '$200' and convert it to a float
        newdf = self.processed_data.copy()
        for var in variables:
            if newdf[var].dtypes == 'object': 
                # if it was a string, convert variable to float 
                newdf[var] = newdf[var].apply(lambda x: float(x.split("$")[1].replace(",","")) if type(x) == str else x)
            else: 
                pass
        self.processed_data = newdf


    def fillMissing(self):  
        # fill NaN with mean and mode
        df = self.processed_data.copy()
        vars = df.columns
        for var in vars: 
            if any(df[var].isna()):
                if df[var].dtypes == 'float64':
                    # substitute NaN with mean
                    df[var].fillna(value=df[var].mean(), 
                                        inplace=True)
                else:
                    # substitute NaN with mode
                    df[var].fillna(value=df[var].mode()[0], 
                                        inplace=True)
        self.processed_data = df
                    
    def scaling(self, catVars, target):
        # setup scaler
        scaler = StandardScaler()
        df = self.processed_data.copy()

        scaleCols = list(set(df.columns)-set(catVars+[target]))
        # remove both target variables because in the test data they are not provided
        # training set scaling
        df[scaleCols] = scaler.fit_transform(df[scaleCols])        # re-add categorical variables
        # df = df.join(df[catVars])
        self.processed_data = df

    def cat2int(self, variables):
        # Convert categorical variables to numerical
        # Create a copy of the original dataframe to avoid warnings
        df = self.processed_data.copy()
        firstidx = df.index[0]
        for var in variables:
            if type(df[var][firstidx]) == str:
                df[var] = df[var].apply(lambda x: 1 if 'yes' in x.lower() else 0)
                c = 1
            else:
                # if we run the code again it will
                pass
        self.processed_data = df
    
    
    def convertCategorical(self):
        catVar = ['PARENT1', 'MSTATUS', 'RED_CAR', 'REVOKED']  
        # Convert categorical to dummies
        self.cat2int(catVar)
        
        df = self.processed_data.copy()
        # Convert binary to dummies
        # converting SEX
        df['GENDER'] = df['SEX'].apply(lambda x: 1 if 'M' in x else 0)
        df.drop(['SEX'],axis=1,inplace=True)

        # converting CAR_USE, URBANICITY
        df['COMMERCIAL_CAR_USE'] = df['CAR_USE'].apply(lambda x: 1 if 'commercial' in x.lower() else 0)
        df.drop(['CAR_USE'],axis=1,inplace=True)

        df['URBAN_CAR'] = df['URBANICITY'].apply(lambda x: 1 if 'urban' in x.lower() else 0)
        df.drop(['URBANICITY'],axis=1,inplace=True)

        # Convert EDUCATION, JOB, CAR_TYPE
        df['EDUCATION'] = df['EDUCATION'].apply(lambda x: 'Elementary Education' if '<high school' in x.lower() else x)
        # Insert dummy variables and Drop the original variable
        df = df.join(pd.get_dummies(df.EDUCATION.str.upper())).drop(['EDUCATION'],axis=1)
        df = df.join(pd.get_dummies(df.JOB.str.upper())).drop(['JOB'],axis=1)
        df = df.join(pd.get_dummies(df.CAR_TYPE.str.upper())).drop(['CAR_TYPE'],axis=1)

        df.columns = df.columns.str.replace("Z_","")
        df.columns = df.columns.str.replace(" ","_")
        
        self.processed_data = df

In [19]:
def preprocessing(df,train=False):
    print('\n'+'*'*10 + 'Preprocessing' + '*'*10)
    # define a Preprocessor object
    preprocess = Preprocessor(df)

    # Remove useless columns from the dataframes
    toDrop = ['TARGET_AMT'] if train else ['TARGET_FLAG','TARGET_AMT']
    toDrop += ['INDEX']
    preprocess.dropVar(toDrop)
    cat2numVars = ["INCOME","HOME_VAL","BLUEBOOK","OLDCLAIM"]
    preprocess.str2int(cat2numVars)
    
    # return preprocess.processed_data

    catcols = ['PARENT1', 'MSTATUS', 'RED_CAR', 'REVOKED',
                   'SEX','CAR_USE','URBANICITY','JOB','CAR_TYPE','EDUCATION']

    preprocess.scaling(catcols,target='TARGET_FLAG')
    preprocess.convertCategorical()
    preprocess.fillMissing()


    return preprocess.processed_data

In [18]:
#read data from GCS
import pandas as pd
df_train = pd.read_csv("gs://gcp-workshop-input/train_auto.csv")
df_test = pd.read_csv("gs://gcp-workshop-input/test_auto.csv")

In [20]:
pretrain = preprocessing(df_train,True)
pretest  = preprocessing(df_test,False)


**********Preprocessing**********

**********Preprocessing**********


## we now save the preprocessed data to cloud storage

In [40]:
pretrain.to_csv(f"gs://{bucket_name}/auto_clean_train.csv", index=False)

## and transfer that to bigquery

In [44]:
TABLE_ID = 'train_clean'
table_id = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

job_config = bigquery.LoadJobConfig(autodetect=True,
                                    skip_leading_rows=1,
    # The source format defaults to CSV, so the line below is optional.
                                    source_format=bigquery.SourceFormat.CSV,
                                   )

file_name = "auto_clean_train.csv"
uri = f"gs://{bucket_name}/{file_name}"

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 8161 rows.


In [21]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
TARGET = 'TARGET_FLAG'
X_train,y_train = pretrain[pretrain.columns.drop(TARGET)], pretrain[TARGET]
rf.fit(X_train, y_train)

RandomForestClassifier()

In [22]:
pd.Series(rf.predict(pretest)).value_counts()

0    1824
1     317
dtype: int64

In [23]:
pd.Series(rf.predict(pretrain[pretrain.columns.drop(TARGET)])).value_counts()

0    6008
1    2153
dtype: int64

### TRAIN WITH bQ ML

In [73]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()
     

In [74]:
MODEL_ID = 'autoinsurance_randomforest' + UUID
print(MODEL_ID)

autoinsurance_randomforestoo2tk3mu


In [107]:
modelq= f'''CREATE MODEL `{PROJECT_ID}.{DATASET_ID}.{MODEL_ID}`
OPTIONS(MODEL_TYPE='RANDOM_FOREST_CLASSIFIER',
        NUM_PARALLEL_TREE = 50,
        TREE_METHOD = 'HIST',
        EARLY_STOP = FALSE,
        SUBSAMPLE = 0.85,
        INPUT_LABEL_COLS = ['{TARGET}'],
        MODEL_REGISTRY = 'VERTEX_AI'
        )        
AS SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`;'''

In [108]:
print(modelq)

CREATE MODEL `sandbox-mvp-001.gcp_workshop.autoinsurance_randomforestoo2tk3mu`
OPTIONS(MODEL_TYPE='RANDOM_FOREST_CLASSIFIER',
        NUM_PARALLEL_TREE = 50,
        TREE_METHOD = 'HIST',
        EARLY_STOP = FALSE,
        SUBSAMPLE = 0.85,
        INPUT_LABEL_COLS = ['TARGET_FLAG'],
        MODEL_REGISTRY = 'VERTEX_AI'
        )        
AS SELECT * FROM `sandbox-mvp-001.gcp_workshop.train_clean`;


In [110]:
job = client.query(modelq)
print(f"Job state: {job.state}\nJob Error:{job.errors}")     

Job state: RUNNING
Job Error:None


In [111]:
job.result()
print(job.state)

DONE


## read model from model registry

In [118]:
from google.cloud import aiplatform
model = aiplatform.Model(MODEL_ID,
                          project=PROJECT_ID, 
                          location = REGION)

## endpoint creation

In [252]:
REGION

'europe-west4'

In [253]:
ENDPOINT_DISPLAY_NAME = MODEL_ID + UUID

endpoint = aiplatform.Endpoint.create(
    display_name=ENDPOINT_DISPLAY_NAME,
    project=PROJECT_ID,
    location=REGION,
)

print(endpoint.display_name)
print(endpoint.resource_name)

Creating Endpoint


INFO:google.cloud.aiplatform.models:Creating Endpoint


Create Endpoint backing LRO: projects/736100380721/locations/europe-west4/endpoints/451881686829891584/operations/6339737316512235520


INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/736100380721/locations/europe-west4/endpoints/451881686829891584/operations/6339737316512235520


Endpoint created. Resource name: projects/736100380721/locations/europe-west4/endpoints/451881686829891584


...

To use this Endpoint in another session:


INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:


endpoint = aiplatform.Endpoint('projects/736100380721/locations/europe-west4/endpoints/451881686829891584')


INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/736100380721/locations/europe-west4/endpoints/451881686829891584')


autoinsurance_randomforestoo2tk3muoo2tk3mu
projects/736100380721/locations/europe-west4/endpoints/451881686829891584


## model deploy to endpoint

In [144]:
MODEL_ID

'autoinsurance_randomforestoo2tk3mu'

In [121]:
DEPLOYED_NAME = MODEL_ID

model.deploy(endpoint=endpoint, deployed_model_display_name=DEPLOYED_NAME)

print(model.display_name)
print(model.resource_name)

Deploying model to Endpoint : projects/736100380721/locations/europe-west4/endpoints/353928394934583296


INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/736100380721/locations/europe-west4/endpoints/353928394934583296


Using default machine_type: n1-standard-2


INFO:google.cloud.aiplatform.models:Using default machine_type: n1-standard-2


Deploy Endpoint model backing LRO: projects/736100380721/locations/europe-west4/endpoints/353928394934583296/operations/5488275511962501120


INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/736100380721/locations/europe-west4/endpoints/353928394934583296/operations/5488275511962501120


Endpoint model deployed. Resource name: projects/736100380721/locations/europe-west4/endpoints/353928394934583296


INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/736100380721/locations/europe-west4/endpoints/353928394934583296


autoinsurance_randomforestoo2tk3mu
projects/736100380721/locations/europe-west4/models/autoinsurance_randomforestoo2tk3mu


## use the endpoint for online predictions

In [275]:
PROJECT_ID,REGION

('sandbox-mvp-001', 'europe-west4')

In [315]:
display = 'autoinsurance_randomforestoo2tk3muoo2tk3mu'
endpoint_list = aiplatform.Endpoint.list(filter = f'''display_name = "{display}"'''
                                     ,project=PROJECT_ID,
                                     location=REGION,)
# endpoint.gca_resource

In [322]:
endpoint_list

 resource name: projects/736100380721/locations/europe-west4/endpoints/353928394934583296]

In [318]:
project = '736100380721'
location = 'europe-west4'
endpoint_id = '353928394934583296'
endpoint = aiplatform.Endpoint(
    endpoint_name=endpoint_id,
    project=project,
    location=location)

In [320]:
path = 'gs://gcp-workshop-vertex-pipeline/pipeline_root/736100380721/pipeline-predict-workshop-20230228135915/preprocess_1193498981229920256/dataset_processed.csv'

In [321]:
dfpipe = pd.read_csv(path)
instances = dfpipe.to_dict(orient='records')
prediction = endpoint.predict(instances)
dfpipe['prediction'] = pd.DataFrame(prediction.predictions)['predicted_TARGET_FLAG']

## bq predictions

In [133]:
sql_ml_predict = f"""SELECT * FROM ML.PREDICT(MODEL `{PROJECT_ID}.{DATASET_ID}.{MODEL_ID}`, 
(SELECT
      *
    FROM
      `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}` LIMIT 10))"""

job = client.query(sql_ml_predict)
prediction_result = job.result().to_arrow().to_pandas()

In [134]:
prediction_result.head()

,predicted_TARGET_FLAG,predicted_TARGET_FLAG_probs,TARGET_FLAG,KIDSDRIV,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,...,MANAGER,PROFESSIONAL,STUDENT,BLUE_COLLAR,MINIVAN,PANEL_TRUCK,PICKUP,SPORTS_CAR,VAN,SUV
0,0,"[{'label': 1, 'prob': 0.223684623837471}, {'la...",0,-0.334421,-0.555266,2.937290,1.099825,-0.290755,1,0.241317,...,0,0,0,0,1,0,0,0,0,0
1,1,"[{'label': 1, 'prob': 0.616367518901825}, {'la...",1,-0.334421,-0.555266,1.145585,0.611091,-0.238831,0,0.301960,...,0,0,0,1,0,0,0,0,1,0
2,0,"[{'label': 1, 'prob': 0.19636750221252441}, {'...",0,-0.334421,-0.555266,2.041438,1.588559,0.720437,0,0.983947,...,1,0,0,0,1,0,0,0,0,0
3,1,"[{'label': 1, 'prob': 0.685663104057312}, {'la...",1,-0.334421,-0.555266,0.249732,0.855458,-1.238776,0,-1.199449,...,0,0,1,0,1,0,0,0,0,0
4,0,"[{'label': 1, 'prob': 0.2368536740541458}, {'l...",0,3.575626,-0.555266,1.145585,0.122358,0.660336,0,0.855287,...,0,0,0,1,0,0,1,0,0,0


## bQ evaluation

In [139]:
INPUT_EVAL = 'train_clean'
eval_q = f'''SELECT
  *
FROM
  ML.EVALUATE (MODEL `{PROJECT_ID}.{DATASET_ID}.{MODEL_ID}`,
    (
    SELECT
      *
    FROM
      `{PROJECT_ID}.{DATASET_ID}.{INPUT_EVAL}`
    )
  )'''

In [141]:
client.query(eval_q).to_dataframe()

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.940949,0.791918,0.931994,0.860025,0.308312,0.962868
